#Cource of Content...

#Module1:###
1. Architecture
2. Create a table
3. Read a table
4. Update a table
6. Delete from a table
7. Upsert to atable
8. Clean up snapshots with VACUUM
9. Display table history.
10. Query an earlier version of the (time travel)
11. Optimize tables
12. Update schema.
13. Selective Overwrite
14. Custom Metadata
15. Change Data Feed
16. Contraints
17. Table Utility Commands
18. Clone Delta tables
19. Convert to Delta Lake    

#Module 2:###
1. End to End project using Delta Lake.
![test_image](/files/tables/Azure-3.png)

#Introduction and Architecture###
![test_image](/files/tables/Azure-1.png)

Enterprises here and now catalyze vast quantities of data, which can be a high-end source of business intelligence and insight when used appropriately. Delta Lake allows businesses to access and break new data down in real time.

Delta Lake is an open-source warehouse layer designed to run on top of data lakes analogous to S3, ADLS, GCS, and HDFS to amend trustability, security, and performance. It supports ACID transactions, extensible metadata, integrated streaming, and batch processing.

Delta Lake is an open source storage layer that enables building a data lakehouse on top of exixting storage systems over cloud objects with additional features like ACID properties, schema enforcement, time travel features enabled.

An open-source spreadsheet format that isn’t vendor-dependent. It’s also saved in the Parquet file format with numerous benefits of warehouse technology and can be distributed anywhere. In addition, it has time travel features, exposes metadata and statistics, and allows for data skipping and z- ordering to enhance query performance.

Learning Objectives

In this article, you will learn about the following:

Delta Lake adds intelligent data governance and control set to an open warehouse medium for structured,semi-structured, and unstructured data, supporting streaming and batch operations from a single source.
The Lakehouse combines the best of the data lake and data warehouse. This is open-source, with high performance, reduced data movement and overall cost, and better support for AI and ML workloads.
The Delta Lake framework runs on top of the current data lake and is exhaustively compatible with the Apache Spark API.
Here are a few reasons why you should be familiar with the Delta Lakes:

--->Delta Lake helps fill multifold gaps in the data warehouse.
--->One unified platform for Data and AI.
--->Combine Data Warehouse Performance with Data Lake Flexibility.
--->Ultramodern Date Lakehouse Architectures with Delta Lake.


#How Delta Lake Implements Transactions
Here is how Delta Lake implements transactions:

Read the existing metadata
Read the existing Parquet data files
Write the Parquet files for the current transaction
Record the new transaction in the transaction log (if there are no conflicts)
Let’s recall our delete operation from the prior section and see how it fits into this transaction model:

We read the existing metadata to find the file paths for the existing Parquet files
We read the existing Parquet files and identify the files that contains data that should be removed
We write new Parquet files with the deleted data filtered out
Once the new Parquet files are written, we check for conflicts and then make an entry in the transaction log. The next section will discuss transaction conflicts in more detail.
Blind append operations can skip a few steps and are executed as follows:

Write the Parquet files for the current transaction
Record the new transaction in the metadata
Delta implements a non locking MVCC (multi version concurrency control) so writers optimistically write new data and simply abandon the transaction if it conflicts at the end. The alternative would be getting a lock at the start thereby guaranteeing the transaction immediately.

Let’s look at the case when a Delta Lake transaction conflicts.

How Delta Lake transactions can conflict
Suppose you have a transaction that deletes a row of data that’s stored in FileA (Transaction 1). While this job is running, there is another transaction that deletes some other rows in FileA (Transaction 2). Transaction 1 finishes running first and is recorded in the metadata.

Before Transaction 2 is recorded as a transaction, it will check the metadata, find that Transaction 2 conflicts with a transaction that was already recorded (from Transaction 1), and error without recording a new transaction.

Transactions 2 will write Parquet data files, but will not be recorded as a transaction, so the data files will be ignored. The zombie Parquet files can be easily cleaned up via subsequent vacuum operations.

Transaction 2 must fail otherwise it would cause the data to be incorrect.

Delta Lake transactions prevent users from making changes that would corrupt the table. Transaction conflict behavior can differ based on isolation level, which controls the degree to which a transaction must be isolated from modifications made by other concurrent transactions. More about this in the concurrency section.

Transactions rely on atomic primitives storage guarantees
Suppose you have two transactions that are finishishing at the same exact time. Both of these transactions look at the existing Delta Lake transaction log, see that the latest transaction was 003.json and determine that the next entry should be 004.json.

If both transactions are recorded in the 004.json file, then one of them will be clobbered, and the transaction log entry for the clobbered metadata entry will be lost.

Delta tables rely on storage systems that provide atomic primitives for safe concurrency. The storage system must allow Delta Lake to write the file, only if it does not exist already, and error out otherwise. The storage system must NOT permit concurrent writers to overwrite existing metadata entries.

Some clouds have filesystems that don’t explicitly support these atomic primitives, and therefore must be coupled with other services to provide the necessary guarantees.

Delta Lake transactions are only for a single table
Delta Lake transactions are only valid for a single table.

Some databases offer transaction support for operations that impact multiple tables. Delta Lake does not support multi-table transactions.

Data lakes don’t support transactions
Data lakes consist of many files in a storage system (e.g. a cloud storage system) and don’t support transactions.

Data lakes don’t have a metadata layer, conflict resolution, or any way to store information about transactions.

Data lakes are prone to multiple types of errors because they don’t support transactions:

Easy to corrupt
Downtime/unstable state while jobs are running
Operations can conflict
Data lakes have many downsides and it’s almost always better to use a lakehouse storage system like Delta Lake compared to a data lake.

#ACID Transactions
![test_image](/files/tables/Azure-2.png)
We’ve already explored how Delta Lake supports transactions. This section explains how Delta Lake transactions have the Atomic, Consistent, Isolated and Durable (ACID transaction) properties. Reading this section is optional.

ACID transactions are commonplace in databases but notably absent for data lakes.

Delta Lake’s ACID transaction support is one of the major reasons it is almost always a better option than a data lake.

Let’s explore how Delta Lake allows for ACID transactions.

###Atomic Transactions

An atomic transaction either fully completes or fully fails, with nothing in between.

Delta Lake transactions are atomic, unlike data lake transactions that are not atomic.

Suppose you have a job that’s writing 100 files to a table. Further suppose that the job errors out and the cluster dies after writing 40 files:

For a Delta table, no additional data will be added to the table. Parquet files were written to the table, but the job errored, so no transaction log entry was added and no data was added to the table.
For a data lake, the 40 files are added and the transaction “partially succeeds”.
For data tables, it’s almost always preferable to have a transaction that “fully fails” instead of one that “partially succeeds” because partial writes are hard to unwind and debug.

Delta Lake implements atomic transactions by writing data files first before making a new entry in the Delta transaction log.

These guarantees are provided at the protocol level through the "transaction" abstraction. We’ve already discussed what constitutes a transaction for Delta Lake.

If there is an error with the transaction and some files don’t get written, then no metadata entry is made and the partial data write is ignored. The zombie Parquet files can be easily cleaned up via subsequent vacuum operations.

Now let’s look at how Delta Lake also provides consistent transactions.

###Consistent Transactions

Consistency means that transactions won’t violate integrity constraints on the Delta table.

Delta Lake has two types of consistency checks:

Schema enforcement checks
Column constraints
Schema enforcement checks verify that new data appended to a Delta table matches the schema of the existing table. You cannot append data with a different schema, unless you enable schema evolution.

Delta Lake column constraints allow users to specify the requirements of data that’s added to a Delta table. For example, if you have an age column with a constraint that requires the value to be positive, then Delta Lake will reject appends of any data that doesn’t meet the constraint.

Data lakes don’t support schema enforcement or column constraints. That’s another reason why data lakes are not ACID-compliant.

###Isolated Transactions

Isolation means that transactions are applied to a Delta table sequentially.

Delta Lake transactions are persisted in monotonically increasing transaction files, as we saw in the previous example. 
First 00000000000000000000.json, 
then 00000000000000000001.json, 
then 00000000000000000002.json, and so on.

Delta Lake uses concurrency control to ensure that transactions are executed sequentially, even when user operations are performed concurrently. The next page of this guide explains concurrency in Delta Lake in detail.

###Durable Transactions

Delta tables are generally persisted in cloud object stores which provide durability guarantees.

Durability means that all transactions that are successfully completed will always remain persisted, even if there are service outages or program crashes.

Suppose you have a Delta table that’s persisted in Azure blob storage. The Delta table transactions that are committed will always remain available, even in these circumstances:

When there are Azure service outages
If a computation cluster that’s writing the Delta table crashes for some reason
Two operations are running concurrently and one of them fails
Successful transactions are always registered in the Delta table and persisted no matter what.

###Conclusion
Delta Lake supports transactions which provide necessary reliability guarantees for production data systems.

Vanilla data lakes don’t provide transactions and this can cause nasty bugs and a bad user experience. Let’s look at a couple of scenarios when the lack of transactions cause a poor user experience:

While running a compaction operation on a data lake, newly compacted “right sized” files are added before the small files are deleted. If you read the data lake while this operation is running, you will see duplicate data.
While writing to a data lake, a job might fail, which leaves behind partially written files. These files are corrupt, which means that the data lake cannot be read until the corrupt files are manually removed.
Users want to run a simple DML operation like deleting a few rows of data which require a few files to be rewritten. This operation renders the data lake unusable until it’s done running.
Transactions are a key advantage of Delta Lake vs. data lakes. There are many other advantages, but proper transactions are necessary in production data environments.

#Delta Lake Architecture Diagram
The Delta Lake Architecture is a massive improvement upon the conventional Lambda architecture.
At each stage, it improves our data through a connected pipeline and allows us to combine streaming and batch workflows through a shared file store with ACID-compliant transactions.

It organizes our data into layers or folders defined as bronze, silver, and gold as follows…

###Bronze 
tables have raw data ingested from various sources (RDBMS data, JSON files, ITT data, etc.)

###Silver 
tables will give a more refined view of our data using joins.

###Gold 
tables give business-level aggregates often used for dashboarding and reporting.
And these Gold Tables can be consumed by various Business Intelligence tools for reporting and analytics purposes.

#Data Lake
###Tables----------->SQL Server
###ADLS Gen2-------->CSV
###S3 Bucket-------->Parquet Files

Source--->Ingest(Raw(Bronze) Layer)--->Cleansed(Silver Layer)--->LOB(Business Layer Gold Layer).

#Databricks Tables vs Normal Tables
Databricks tables and normal tables typically refer to tables within the Databricks Unified Analytics Platform, often used for Apache Spark-based data processing. Here's how they differ:

###Databricks Table:

Managed by Databricks: Databricks tables are managed tables, meaning Databricks takes care of storing metadata and managing the underlying data files. They are typically stored in a distributed file system such as Azure Data Lake Storage (ADLS) or Amazon S3.

Integration with Delta Lake: Databricks tables can be Delta tables, leveraging all the features and benefits of Delta Lake, such as ACID transactions, schema evolution, time travel, and more.

Optimization and Caching: Databricks tables can be optimized and cached in memory for faster query performance. Databricks automatically optimizes queries whenever possible, improving efficiency.

Seamless Integration with Databricks Environment: Databricks tables seamlessly integrate with other Databricks features like notebooks, jobs, and MLflow, providing a unified environment for data processing, analytics, and machine learning.

###Normal Table:

External Tables: Normal tables in Databricks are often external tables, meaning the data files are stored externally, and Databricks only manages the metadata. These tables are not managed by Databricks in terms of data storage.

May Lack Delta Lake Features: Normal tables may lack the advanced features provided by Delta Lake, such as ACID transactions, schema enforcement, and time travel. They rely on the capabilities of the underlying storage system.

Manual Management: With normal tables, you may need to manage data files and metadata manually, including loading, updating, and deleting files.

Limited Optimization: While you can still optimize queries on normal tables, they may not benefit from the same level of optimization as Databricks tables, particularly when it comes to caching and query planning.

In summary, Databricks tables offer a more integrated and managed approach within the Databricks environment, often leveraging the capabilities of Delta Lake for enhanced reliability and performance. Normal tables, on the other hand, provide more flexibility but may require more manual management and lack some of the advanced features available with Databricks tables.

#How Delta Tables differs from Normal Tables


###Parquet Table

In [0]:
%sql
create database if not exists deltalake;

In [0]:
%sql
use deltalake;

In [0]:
%sql
show databases;

databaseName
default
deltalake


In [0]:
%sql
show tables;

database,tableName,isTemporary


In [0]:
%sql 
create table if not exists test (id int) using parquet;

In [0]:
%sql
drop table if exists test;

In [0]:
%sql
desc extended test;

col_name,data_type,comment
id,int,null
,,
# Detailed Table Information,,
Catalog,spark_catalog,
Database,deltalake,
Table,test,
Owner,root,
Created Time,Fri May 17 09:02:54 UTC 2024,
Last Access,UNKNOWN,
Created By,Spark 3.3.2,


In [0]:
%sql 
insert into test values(1),(2)

In [0]:
%sql 
update test set id=3 where id=2

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-3503008565651212>:7
      5     display(df)
      6     return df
----> 7   _sqldf = ____databricks_percent_sql()
      8 finally:
      9   del ____databricks_percent_sql

File <command-3503008565651212>:4, in ____databricks_percent_sql()
      2 def ____databricks_percent_sql():
      3   import base64
----> 4   df = spark.sql(base64.standard_b64decode("dXBkYXRlIHRlc3QxIHNldCBpZD0zIHdoZXJlIGlkPTI=").decode())
      5   display(df)
      6   return df

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(
     50         module_name, class_name, function_name, time.perf_counter() - start, signature
     51     )
     52     return res

Fil

#Delta Table


###Managed Tables(Internal Tables)

In [0]:
%sql 
create table if not exists deltalake.practice (id int) using delta;

In [0]:
%sql
desc extended practice;

col_name,data_type,comment
id,int,null
,,
# Detailed Table Information,,
Catalog,spark_catalog,
Database,deltalake,
Table,practice,
Created Time,Fri May 17 09:06:42 UTC 2024,
Last Access,UNKNOWN,
Created By,Spark 3.3.2,
Type,MANAGED,


In [0]:
%sql 
insert into practice values(1),(2)

num_affected_rows,num_inserted_rows
2,2


In [0]:
%sql
select * from practice;

id
1
2


In [0]:
%sql 
update practice set id=3 where id=2

num_affected_rows
1


In [0]:
%sql
select * from practice;

id
1
3


In [0]:
dbutils.fs.ls('dbfs:/user/hive/warehouse/deltalake.db/practice')

Out[26]: [FileInfo(path='dbfs:/user/hive/warehouse/deltalake.db/practice/_delta_log/', name='_delta_log/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/user/hive/warehouse/deltalake.db/practice/part-00000-4b36fa40-57f6-4984-a7b2-51487f77aea5-c000.snappy.parquet', name='part-00000-4b36fa40-57f6-4984-a7b2-51487f77aea5-c000.snappy.parquet', size=579, modificationTime=1715936826000),
 FileInfo(path='dbfs:/user/hive/warehouse/deltalake.db/practice/part-00000-9c4a355f-770c-4615-acbf-e1fbf60e284f-c000.snappy.parquet', name='part-00000-9c4a355f-770c-4615-acbf-e1fbf60e284f-c000.snappy.parquet', size=579, modificationTime=1715936849000)]

In [0]:
dbutils.fs.ls('dbfs:/user/hive/warehouse/deltalake.db/test')

Out[27]: [FileInfo(path='dbfs:/user/hive/warehouse/deltalake.db/test/_SUCCESS', name='_SUCCESS', size=0, modificationTime=1715936638000),
 FileInfo(path='dbfs:/user/hive/warehouse/deltalake.db/test/_committed_5705977496109481042', name='_committed_5705977496109481042', size=224, modificationTime=1715936638000),
 FileInfo(path='dbfs:/user/hive/warehouse/deltalake.db/test/_started_5705977496109481042', name='_started_5705977496109481042', size=0, modificationTime=1715936638000),
 FileInfo(path='dbfs:/user/hive/warehouse/deltalake.db/test/part-00000-tid-5705977496109481042-a5188d56-3564-43a8-baeb-d86fd297738f-198-1-c000.snappy.parquet', name='part-00000-tid-5705977496109481042-a5188d56-3564-43a8-baeb-d86fd297738f-198-1-c000.snappy.parquet', size=568, modificationTime=1715936638000),
 FileInfo(path='dbfs:/user/hive/warehouse/deltalake.db/test/part-00001-tid-5705977496109481042-a5188d56-3564-43a8-baeb-d86fd297738f-199-1-c000.snappy.parquet', name='part-00001-tid-5705977496109481042-a5188d56

###External Location


In [0]:
%sql 
create table if not exists deltalake.test3 (id int) location '/mnt/FileStore/deltalake';

In [0]:
%sql
desc extended test3;

col_name,data_type,comment
id,int,null
,,
# Detailed Table Information,,
Catalog,spark_catalog,
Database,deltalake,
Table,test3,
Created Time,Fri May 17 09:08:49 UTC 2024,
Last Access,UNKNOWN,
Created By,Spark 3.3.2,
Type,EXTERNAL,


#Create a Delta Table

###With SQL

In [0]:
%sql
create table if not exists employee
(
  id int,
  Name string
)
using delta

In [0]:
%sql
desc extended employee;

col_name,data_type,comment
id,int,null
Name,string,null
,,
# Detailed Table Information,,
Catalog,spark_catalog,
Database,deltalake,
Table,employee,
Created Time,Fri May 17 09:09:24 UTC 2024,
Last Access,UNKNOWN,
Created By,Spark 3.3.2,


In [0]:
%sql
create table if not exists employee1
(
  id int,
  Name string
)
using delta
location '/mnt/deltalake/employee1'

In [0]:
%sql
desc extended employee1;

col_name,data_type,comment
id,int,null
Name,string,null
,,
# Detailed Table Information,,
Catalog,spark_catalog,
Database,deltalake,
Table,employee1,
Created Time,Fri May 17 09:09:54 UTC 2024,
Last Access,UNKNOWN,
Created By,Spark 3.3.2,


In [0]:
%python
spark.sql("""create table if not exists employee2
(
  id int,
  name string
)
using delta""")

Out[35]: DataFrame[]

In [0]:
%sql
desc extended employee2;

col_name,data_type,comment
id,int,null
name,string,null
,,
# Detailed Table Information,,
Catalog,spark_catalog,
Database,deltalake,
Table,employee2,
Created Time,Fri May 17 09:10:17 UTC 2024,
Last Access,UNKNOWN,
Created By,Spark 3.3.2,


In [0]:
display(_sqldf)

col_name,data_type,comment
id,int,null
name,string,null
,,
# Detailed Table Information,,
Catalog,spark_catalog,
Database,deltalake,
Table,employee2,
Created Time,Fri May 17 09:10:17 UTC 2024,
Last Access,UNKNOWN,
Created By,Spark 3.3.2,


###With Python

In [0]:
from delta.tables import *
DeltaTable.createIfNotExists(spark).\
    tableName("employee3").\
    addColumn("ID","Int").\
    addColumn("Name", "String", comment="This is student Name").\
    execute()

Out[38]: <delta.tables.DeltaTable at 0x7f1c0ef8a370>

In [0]:
%sql
desc extended employee3

col_name,data_type,comment
ID,int,null
Name,string,This is student Name
,,
# Detailed Table Information,,
Catalog,spark_catalog,
Database,deltalake,
Table,employee3,
Created Time,Fri May 17 09:10:53 UTC 2024,
Last Access,UNKNOWN,
Created By,Spark 3.3.2,


In [0]:
from delta.tables import *
DeltaTable.createOrReplace(spark).\
    addColumn("ID","Int").\
    addColumn("Name", "String", comment="This is student Name").\
    location('/mnt/deltalake/employee4').\
    execute()

Out[40]: <delta.tables.DeltaTable at 0x7f1c0f02c6a0>

###Data which is partiton level
Partitions can be beneficial, especially for very large tables. Many performance enhancements around partitioning focus on very large tables (hundreds of terabytes or greater). Many customers migrate to Delta Lake from Parquet-based data lakes.
![test-image](files/tables/Azure-5.png)
/mnt/deltalake/jan/===>

###College
![test-image](/files/tables/Azure-4.png)
/mnt/deltalake/college/deb=EEE
/mnt/deltalake/college/deb=IT
/mnt/deltalake/college/deb=CSE
/mnt/deltalake/college/deb=ECE
/mnt/deltalake/college/deb=MECH
/mnt/deltalake/college/deb=CIVIL
/mnt/deltalake/college/deb=AERO


In [0]:
%python
sql("drop table employee5")
spark.sql(""" create table if not exists employee5
          (id int,
          Name string, 
          Department string
          )
          using delta""")

Out[42]: DataFrame[]

In [0]:
%sql 
desc extended employee5

col_name,data_type,comment
id,int,null
Name,string,null
Department,string,null
,,
# Detailed Table Information,,
Catalog,spark_catalog,
Database,deltalake,
Table,employee5,
Created Time,Fri May 17 09:12:26 UTC 2024,
Last Access,UNKNOWN,
